In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG = True

from collections import Counter
import parmed as pmd

# Informatics

ParmEd parses a lot of information out of a PDB or PDBx/mmCIF file.

In [ ]:
pdb = pmd.download_CIF("2koc")

In [ ]:
pdb.experimental

In [ ]:
pdb.pmid, pdb.doi

In [ ]:
pdb.journal, pdb.page, pdb.year, pdb.journal_authors

In [ ]:
pdb.related_entries

# Automatic file type determination

File type is determined by the contents rather than filename extension.

For many file types you can also specify a coordinate file to be loaded alongside.

In [ ]:
parm = pmd.load_file("amber.parm7", "amber.rst7")
parm

# System Manipulation

ParmEd supports a rich selection syntax, and selected subsystems remain fully parametrized (only parameters involving "removed" atoms are lost).

### Amber Mask selection

In [ ]:
parm["@CA,C,N"]

In [ ]:
parm["!:WAT"]

In [ ]:
parm[":1-9"]

### Atom slice selection

The `Structure` type and its subclasses also behave as either a 1-axis list of atoms or a 2-axis list of residues/atoms.

In [ ]:
parm[:119]

In [ ]:
parm[:9, :]

In [ ]:
parm[:9, :2]

In [ ]:
# A single index will return a single atom
parm[0]

## Slices vs. views

Slices create an entirely new copy. Editing the resulting structure will not impact the original.

In [ ]:
new_parm = parm[:]

for atom in new_parm[:119]:
    atom.charge = 0
max(a.charge for a in new_parm[:119])

There is a special `view` attribute that exposes a `StructureView` that can be sliced and accessed the same way a `Structure` can, but does *not* return a new instance.

In [ ]:
for atom in new_parm.view[:119]:
    atom.charge = 0
max(a.charge for a in new_parm[:119])

# Bringing ParmEd Actions into the mix

Any action you can execute in the `parmed` CLI can also be imported from `parmed.tools.actions` and used directly.

In [ ]:
new_parm = parm[:]

Counter(a.mass for a in new_parm.atoms if a.element == 1)

In [ ]:
pmd.tools.actions.HMassRepartition(new_parm, 3.024, ring_hmass=2.016).execute()

Counter(a.mass for a in new_parm.atoms if a.element == 1)

In [ ]:
sum(a.mass for a in parm.atoms), sum(a.mass for a in new_parm.atoms)

# Fun with Pandas

![panda]()

If you are comfortable with pandas, you can export a `Structure` as a pandas DataFrame and manipulate the resulting data.

In [ ]:
df = parm.to_dataframe()
df

There is also an option to load atomic data from the DataFrame back to the original `Structure`

In [ ]:
parm.atoms[:10]

In [ ]:
df["name"] = "AMBER"
new_parm = parm[:]
new_parm.load_dataframe(df)
new_parm.atoms[:10]

# Save the resulting structure

And of course you can save the resulting structure to any file format

In [ ]:
parm.save("system.top", format="GROMACS")
parm.save("system.gro")